In [1]:
%%capture --no-stderr
%pip install --quiet -U chromadb

In [2]:
import os, getpass

def _set_env(var: str):
    # Check if the variable is set in the OS environment
    env_value = os.environ.get(var)
    if not env_value:
        # If not set, prompt the user for input
        env_value = getpass.getpass(f"{var}: ")
    
    # Set the environment variable for the current process
    os.environ[var] = env_value

_set_env("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "soeur-modular-rag"

## Modular RAG Orchestration

In [3]:
from pre_retrieval_metadata import extract_metadata
from product_retriever import ProductQuery, retrieve_products, retrieve_product_by_ids
from purchase_history import get_product_ids_by_customer, extract_fashion_preferences
from pre_retrieval_query_transformation import remove_price_from_query
from response_generation import generate_response
import config

def recommend_products(customer_id, query):
    """Search for products and personalize recommendation."""

    # Create the query object with the original query
    product_query = ProductQuery(query)

    # --- Pre-retrieval ---
    
    # Extract price metadata for hybrid retrieval, if enabled
    if config.enable_metadata_extraction:
        product_query = extract_metadata(product_query)

    # Transforms the original query string to remove price information, if enabled
    if config.enable_query_transformation:
        product_query = remove_price_from_query(product_query)

    # --- Retrieval ---
    
    # Retrieve products
    print("\n---Query for retrieval---\n")
    print(product_query)
    contents = retrieve_products(product_query)
    search_results = "\n\n".join(contents)
    print("\n---Retrieval results---\n")
    print(search_results)
    
    # Apply personalization if enabled, identifying customer preferences from the fashion history
    customer_preferences = None
    if config.enable_purchase_history:
        customer_preferences = extract_fashion_preferences(customer_id)
        print("\n---Customer preferences---\n")
        print(customer_preferences)

    # --- Generation ---
    
    # Generate the response
    return generate_response(product_query, search_results, customer_preferences)

# Example usage
customer_id = 3
query = "Elegant navy evening gown below 250"
response = recommend_products(customer_id, query)
print("\n---Final Response---\n")
print(response)


---Query for retrieval---

ProductQuery(query='Elegant navy evening gown', metadata={'price_amount': '250', 'comparison_operator': '$lt', 'category': 'Dresses'})

---Retrieval results---

ella dress Navy blue draped organic cotton dress    BLUE ELLA DRESS|245.00 €|- Fitted cut- Mid-calf length- High collar with buttoned opening- Invisible zip opening with small hook- Front slit at the bottom- Draped detail- Very lightweight cotton poplin The model is 1m77(5'9)  tall and wears a size  38.Reference: ROB1650ELLA25SBLE19|100% ORGANIC COTTON Made in Tunisia

akiko dress Light wool gabardine shirt dress    AKIKO NAVY DRESS|99.00 €|AKIKO is a shirt-dress made from lightweight wool gabardine. The removable belt allows it to be worn straight or to cinch in the waist.- Straight cut, shirt style- Long sleeves with button tabs on the cuffs.- Round neckline with stand collar- Buttoned opening and removable belt- Large patch pocket on the chest- Shirttail hem trousers- Lightweight Wool Gabardine Th